In [1]:
import pygame
import random
import math
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
conveyor_belts = [
    [(1200, 600), (100, 600)],
    [(100, 550), (800, 550)],
    [(200, 500), (800, 500)],
    [(200, 450), (700, 450)],
    [(300, 400), (700, 400)],
    [(300, 350), (600, 350)],
    [(100, 250), (900, 250)],
    [(100, 600), (100, 200)],
    [(200, 500), (200, 200)],
    [(300, 400), (300, 200)],
    [(600, 350), (600, 200)],
    [(700, 450), (700, 200)],
    [(800, 550), (800, 200)],
    [(900, 600), (900, 250)],
]

points = [
    (100, 250), (100, 550), (100, 600),
    (200, 250), (200, 450), (200, 500),
    (300, 250), (300, 350), (300, 400),
    (600, 250), (600, 350), (700, 250),
    (700, 400), (700, 450), (800, 250),
    (800, 500), (800, 550), (900, 250),
    (900, 600)
]

pickup_gates = [
    (100, 200),  # Gate 7
    (200, 200),  # Gate 5
    (300, 200),  # Gate 3
    (600, 200),  # Gate 2
    (700, 200),  # Gate 4
    (800, 200),  # Gate 6
]

In [5]:
class Bag:
    
    def __init__(self, bag_id, reservation_id, start_point=(1200,600)):
        self.bag_id = bag_id
        self.reservation_id = reservation_id
        self.position = start_point  # Current position of the bag
        self.current_target = start_point
        self.speed = 1.5  
        self.path = [] 
        self.ready = False

    
    def set_ready(self):
        self.ready = True   
        
    def set_next_target(self, points_map, flag): 

        options = points_map.get(self.reservation_id, [])
    
        if flag and options:
            self.current_target = options.pop(0)  # Get the next point in the sequence
        elif not flag and options:
            self.current_target = options.pop(0)  # Alternate logic, could be the same
        else:
            self.current_target = None  # No more targets, bag stops moving


    def move(self, points_map, flag):

        # If there's no current target, stop the bag
        if not self.current_target:
            return

        # If the bag is at the target, set the next one
        if self.position == self.current_target:
            self.set_next_target(points_map, flag)
            return

        # Calculate movement toward the target
        x1, y1 = self.position
        x2, y2 = self.current_target

        dx = x2 - x1
        dy = y2 - y1
        distance = math.sqrt(dx**2 + dy**2)

        if distance > 0:
            dx /= distance
            dy /= distance

        # Update position
        new_x = x1 + dx * self.speed
        new_y = y1 + dy * self.speed

        # If close to the target, snap to it
        if abs(new_x - x2) < self.speed and abs(new_y - y2) < self.speed:
            self.position = self.current_target
            self.set_next_target(points_map, flag)  # Automatically get the next target
        else:
            self.position = (new_x, new_y)


In [7]:
class Reservation:
    
    def __init__(self, reservation_id):
        self.reservation_id = reservation_id
        self.bags = []  
        self.ready = False  

    def add_bag(self, bag):
        if isinstance(bag, Bag):
            self.bags.append(bag)
            bag.reservation = self
            self.check_ready_status()

    def check_ready_status(self):
        for i in range(0, len(self.bags)):
            if(self.bags[i].ready == False):
                return
        self.ready = True 
        
    def list_bags(self):
        if not self.bags:
            return "No bags added yet."
        return "\n".join(str(bag) for bag in self.bags)


In [9]:
class PickupGate:
    
    def __init__(self, name, capacity):
        self.name = name 
        self.capacity = capacity  
        self.load = 0  
        self.reservations = [None] * 3  
        
    def add_reservation(self, reservation):
        for i in 3:
            if self.reservations[i] is None:  
                self.reservations[i] = reservation
                return True
        print(f"All slots at Gate {self.name} are full! Cannot add reservation.")
        return False


    def find_reservation(self, reservation):
        for i, r in enumerate(self.reservations):
            if r == reservation:
                return i
        return None

    def add_luggage(self, reservation, count=1):

        # Check if the reservation already exists
        reservation_index = self.find_reservation(reservation)
        if reservation_index is not None:
            self.load += count
            return True

        # If the reservation doesn't exist, create a new one if space is available
        new_reservation_index = self.add_reservation(reservation)
        if new_reservation_index is not None:
            # Add luggage for the new reservation
            if self.load + count <= self.capacity:
                self.load += count
                return True
            else:
                print(f"Gate {self.name} is full! Cannot add luggage.")
                # Remove the reservation if luggage cannot be added
                self.reservations[new_reservation_index] = None
                return False

        print(f"Gate {self.name} has no space for a new reservation!")
        return False

    def remove_luggage(self, count=1):
        if self.load - count >= 0:
            self.load -= count
        else:
            print(f"Gate {self.name} does not have enough luggage to remove!")


In [11]:
def visualize_bags_and_conveyor(conveyor_belts, points, bags, points_map):
    pygame.init()
    screen = pygame.display.set_mode((1600, 1000))
    clock = pygame.time.Clock()
    running = True

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Fill the screen with white
        screen.fill((255, 255, 255))

        # Draw conveyor belts
        for path in conveyor_belts:
            for i in range(len(path) - 1):
                pygame.draw.line(screen, (0, 0, 0), path[i], path[i + 1], 5)

        # Draw points
        for point in points:
            pygame.draw.circle(screen, (255, 255, 0), point, 5)

        for bag in bags:
            pygame.draw.circle(screen, (0, 255, 255), (int(bag.position[0]), int(bag.position[1])), 7)

            if bag.position == bag.current_target:
                bag.set_next_target(points_map, True)  # Update to next target dynamically

            bag.move(points_map, True)
            
        font = pygame.font.Font(None, 24) 
        for i, gate in enumerate(pickup_gates):
            pygame.draw.rect(screen, (0, 255, 0), (gate[0] - 10, gate[1] - 10, 20, 20))  # Green rectangles
            #screen.blit(label, (gate[0] - 20, gate[1] - 25))  # Position above the gate
            
        # Update the screen
        pygame.display.flip()

        # Run at 60 FPS
        clock.tick(60)

    pygame.quit()



points_map = {
    
    
    7: [(1200, 600), (100, 600), (100, 550), (100, 250)],
    6: [(1200, 600), (100, 600), (100, 550), (800, 550), (800, 500), (800, 250)], 
    5: [(1200, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (200, 250)], 
    4: [(1200, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450), 
        (700, 400), (700, 250)],
    3: [(1200, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450), 
        (700, 400), (300, 400), (300, 350), (300, 250)],
    2: [(1200, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450), 
        (700, 400), (300, 400), (300, 350), (600, 350), (600, 250)],
    1: [(900, 250), (900,600)]
}

bags = [
    Bag(bag_id=1, reservation_id=6),
    Bag(bag_id=2, reservation_id=3),
]

visualize_bags_and_conveyor(conveyor_belts, points, bags, points_map)


2025-02-01 21:55:08.992 python[32107:1490452] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-01 21:55:08.992 python[32107:1490452] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [21]:

def process_reservations(json_data):
    data = json.loads(json_data)
    reservations = {}
    
    # Create Reservation objects
    for res_data in data['reservations']:
        reservation = Reservation(res_data['reservation_id'])
        reservations[reservation.reservation_id] = reservation
    
    # Create Bag objects and assign them to reservations
    bags = []
    for bag_data in data['bags']:
        reservation = reservations[bag_data['reservation_id']]
        bag = Bag(bag_data['bag_id'], reservation)
        reservation.add_bag(bag)
        bags.append(bag)
    
    return bags, reservations



# Example JSON data (this would come from your JSON input)
json_data = """
{
    "reservations": [
        {"reservation_id": 1},
        {"reservation_id": 2}
    ],
    "bags": [
        {"bag_id": 101, "reservation_id": 1},
        {"bag_id": 102, "reservation_id": 2}
    ]
}
"""

# Parse and process the reservations and bags
bags, reservations = process_reservations(json_data)

NameError: name 'Bag' is not defined